In [1]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [ ]:
#import os.path

#taxi_file = 'https://github.com/df_greenTalksClub/nyc-tlc-df_green/releases/download/green/green_tripdf_green_2019-10.csv.gz'
#filepath = './green_tripdata_2019-10.csv.gz'

#if not os.path.isfile(filepath):
    #!wget $taxi_file 

In [2]:
%config InteractiveShell.ast_node_interactivity = 'all'

import pandas as pd 

filepath = 'green_tripdata_2019-10.csv.gz'
df_green = pd.read_csv(filepath, low_memory=False)
df_green = df_green[['lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount']]
df_green.shape

/var/folders/qd/s937vysx08bfb134210l2q9h0000gn/T/ipykernel_17933/2500155306.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


(476386, 7)

In [3]:
# for testing
from time import sleep 
import datetime
import random

topic_name = 'green-trips'
count = 0
#df_green = df_green[:100]

t0 = time.time()

for row in df_green.itertuples(index=False):
    count += 1
    message = row._asdict()
    message['timestamp'] = datetime.datetime.now().__str__()
    time.sleep(random.randrange(0, 10) / 1000)    
    result = producer.send(topic_name, value=message)   
    if count % 100 == 0:
        print(f"Sent {count} records")

print(f'sent a total of {count} records to topic')
t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent 100 records
Sent 200 records
Sent 300 records
Sent 400 records
Sent 500 records
Sent 600 records
Sent 700 records
Sent 800 records
Sent 900 records
Sent 1000 records
Sent 1100 records
Sent 1200 records
Sent 1300 records
Sent 1400 records
Sent 1500 records
Sent 1600 records
Sent 1700 records
Sent 1800 records
Sent 1900 records
Sent 2000 records
Sent 2100 records
Sent 2200 records
Sent 2300 records
Sent 2400 records
Sent 2500 records
Sent 2600 records
Sent 2700 records
Sent 2800 records
Sent 2900 records
Sent 3000 records
Sent 3100 records
Sent 3200 records
Sent 3300 records
Sent 3400 records
Sent 3500 records
Sent 3600 records
Sent 3700 records
Sent 3800 records
Sent 3900 records
Sent 4000 records
Sent 4100 records
Sent 4200 records
Sent 4300 records
Sent 4400 records
Sent 4500 records
Sent 4600 records
Sent 4700 records
Sent 4800 records
Sent 4900 records
Sent 5000 records
Sent 5100 records
Sent 5200 records
Sent 5300 records
Sent 5400 records
Sent 5500 records
Sent 5600 records
S

In [ ]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = (SparkSession.builder.master("local[*]")
         .appName("GreenTripsConsumer")
         .config("spark.jars.packages", kafka_jar_package).getOrCreate())

In [ ]:
spark.stop()

In [ ]:
green_stream = (spark.readStream.format("kafka")
                .option("kafka.bootstrap.servers", "localhost:9092")
                .option("subscribe", "green-trips") 
                .option("startingOffsets", "earliest")
                .option("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
                .load())

In [ ]:
%config InteractiveShell.ast_node_interactivity = 'all'

def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        display(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

In [ ]:
query.stop()

In [ ]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType()) \
    .add("timestamp", types.TimestampType())

In [ ]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*") 

In [8]:
from pyspark.sql import functions as F

popular_destinations = green_stream \
    .groupBy('DOLocationID', (F.window(F.col("timestamp"), "5 minutes"))).count().orderBy('count', ascending=False)

In [ ]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

In [ ]:
query.stop()